In [ ]:
import MDAnalysis as mda

In [ ]:
structure = "1UBQ.pdb"

In [ ]:
ubiquitin = mda.Universe(structure)

In [ ]:
#polar donor clash (2.750 Å) ARG A 42 NE ↔ GLN A 49 NE2
gln49 = ubiquitin.select_atoms('resid 49')
arg42 = ubiquitin.select_atoms('resid 42')
#polar acceptor clash (3.035 Å) MET A 1 O ↔ VAL A 17 O
met1 = ubiquitin.select_atoms('resid 1')
val17 = ubiquitin.select_atoms('resid 17')

In [ ]:
import MDAnalysis.topology.tables as tables
import MDAnalysis.lib.distances as distances


In [ ]:
tables.vdwradii

In [ ]:
#takes atoms = AtomGroup.atoms, positions = AtomGroup.atoms.positions
#For detecting clashes, cutoff values of 0.4-1.0 Å and allowance values of 0.2-0.6 Å are generally reasonable.

def check_for_clashes(atoms, cutoff=0.6, allowance=0.4, vdwradii=None):

    clashes = set()
    positions = atoms.positions
    
    assert len([a for a in atoms if a]) == positions.shape[0]

    if not vdwradii:
        vdwradii = tables.vdwradii
        
    for a in atoms:
        if not a.type in vdwradii.keys(): 
            print a.type + " has no defined vdw radius"
            return clashes

    # 1-D vector of the upper-triangle of all-to-all distance matrix    
    dist = distances.self_distance_array(positions)
    N = len(positions)
    
    pairs = list()
    [[pairs.append((i,j)) for j in range(i + 1, N)] for i in range(N)]
       
    for x, (d, (i, j)) in enumerate(zip(dist, pairs)):
        a1, a2 = atoms[i], atoms[j]
        r1, r2 = vdwradii[a1.type], vdwradii[a2.type]
        overlap = (r1 + r2) - dist[x] - allowance
        print overlap, r1, r2, dist[x], allowance, cutoff
        if not overlap >= cutoff : continue
        print "CLASH", overlap, dist[x], a1, a2
        clashes.add(frozenset([i+1,j+1]))

    return clashes

In [ ]:
def print_clash(atoms, clashes):
    a1 = atoms[i]
    a2 = atoms[j]
    

In [ ]:
clashes  = set()
clashes.add([1, 8])

In [ ]:
ind = clashes.pop()

In [ ]:
clashes

In [ ]:
notbb = ubiquitin.select_atoms("not backbone")

In [ ]:
val17 = ubiquitin.select_atoms('resid 17')
met1 = ubiquitin.select_atoms('resid 1')
clash1 = met1+val17
gln49 = ubiquitin.select_atoms('resid 49')
arg42 = ubiquitin.select_atoms('resid 42')
clash2 = arg42+gln49

In [ ]:
clash1.select_atoms("not backbone")

In [ ]:
check_for_clashes(clash2, 0.0, 0.0)

In [ ]:
clash2

In [ ]:
import mdanalysis_3dmol as m3d

In [ ]:
viz = m3d.JS3DMol(ubiquitin)
viz.set_style('cartoon')

In [ ]:
viz.set_style('sphere',clash2,spec={'color':'red'})

In [ ]:
clash2.indices